In [2]:
# Import necessary libraries
import cv2  # OpenCV library for computer vision tasks
import numpy as np  # NumPy library for numerical operations
import os  # OS library for interacting with the operating system
import pandas as pd  # Pandas library for data manipulation and analysis
from datetime import datetime  # datetime module for working with dates and times
from sklearn.model_selection import train_test_split  # Function for splitting data into training and testing sets
from tensorflow.keras.models import Sequential  # Keras Sequential model for building neural networks
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense  # Keras layers for building convolutional neural networks

# Function to load images and labels from dataset folder
def load_dataset(dataset_path):
    images = []  # List to store images
    labels = []  # List to store labels
    label_names = {}  # Dictionary to store label names

    for label, name in enumerate(os.listdir(dataset_path)):  # Iterate over folders in the dataset path
        label_names[label] = name  # Store label name in the dictionary
        for image_name in os.listdir(os.path.join(dataset_path, name)):  # Iterate over images in each folder
            image = cv2.imread(os.path.join(dataset_path, name, image_name), cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
            if image is not None:  # Check if the image was loaded successfully
                image = cv2.resize(image, (128, 128))  # Resize the image to 128x128 pixels
                images.append(image)  # Add the image to the list
                labels.append(label)  # Add the label to the list

    return np.array(images), np.array(labels), label_names  # Return images, labels, and label names

# Load dataset
dataset_path = "C:\\Users\\Narthana\\Downloads\\images"  # Path to the dataset folder
images, labels, label_names = load_dataset(dataset_path)  # Load the dataset

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)  # Split data into training and testing sets

# Reshape images
X_train = X_train.reshape(-1, 128, 128, 1)  # Reshape training images for CNN input
X_test = X_test.reshape(-1, 128, 128, 1)  # Reshape testing images for CNN input

# Normalize images
X_train = X_train / 255.0  # Normalize training images to [0, 1] range
X_test = X_test / 255.0  # Normalize testing images to [0, 1] range

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),  # Convolutional layer with 32 filters and 3x3 kernel
    MaxPooling2D(2, 2),  # Max pooling layer with 2x2 pool size
    Conv2D(64, (3, 3), activation='relu'),  # Convolutional layer with 64 filters and 3x3 kernel
    MaxPooling2D(2, 2),  # Max pooling layer with 2x2 pool size
    Flatten(),  # Flatten layer to convert 2D feature maps to 1D feature vector
    Dense(128, activation='relu'),  # Fully connected layer with 128 units and ReLU activation
    Dense(len(label_names), activation='softmax')  # Output layer with units equal to the number of classes and softmax activation
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Compile the model with optimizers and loss function

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))  # Train the model on the training data and validate on the testing data

# Function to register new faces
def register_face():
    name = input("Enter name to register: ")  # Prompt the user to enter a name
    if not os.path.exists(os.path.join(dataset_path, name)):  # Check if the folder for the name exists
        os.makedirs(os.path.join(dataset_path, name))  # Create a new folder for the name
    cap = cv2.VideoCapture(0)  # Initialize the webcam
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')  # Load the pre-trained face detection classifier
    image_count = 0  # Initialize the image count
    while True:
        ret, frame = cap.read()  # Read a frame from the webcam
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))  # Detect faces in the frame
        for (x, y, w, h) in faces:  # Iterate over the detected faces
            image_count += 1  # Increment the image count
            cv2.imwrite(os.path.join(dataset_path, name, f"{name}_{image_count}.jpg"), gray[y:y+h, x:x+w])  # Save the face image with the name and count
            print(f"Image {image_count} captured.")  # Print a message indicating the image capture
            if image_count >= 30:  # Check if 30 images have been captured
                print("Registration successful!")  # Print a success message
                break  # Break out of the loop
        if image_count >= 30:  # Check if 30 images have been captured
            break  # Break out of the loop
    cap.release()  # Release the webcam
    cv2.destroyAllWindows()  # Close any open windows

# Initialize webcam
cap = cv2.VideoCapture(0)  # Initialize the webcam

# Load pre-trained face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')  # Load the pre-trained face detection classifier

# Check if attendance.csv file exists, create a new one if it doesn't
if os.path.exists('attendance.csv'):  # Check if the attendance file exists
    attendance_data = pd.read_csv('attendance.csv')  # Load the attendance data from the file
else:
    attendance_data = pd.DataFrame(columns=['Name', 'Time'])  # Create a new DataFrame with columns 'Name' and 'Time'

quit_program = False
while not quit_program:
    option = input("Choose an option: 1 - Register a new face, 2 - Mark attendance, 3 - Quit: ")  # Prompt the user to choose an option
    if option == '1':
        register_face()  # Call the register_face function
    elif option == '2':
        ret, frame = cap.read()  # Read a frame from the webcam
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))  # Detect faces in the frame
        for (x, y, w, h) in faces:  # Iterate over the detected faces
            face = gray[y:y+h, x:x+w]  # Extract the face region
            face = cv2.resize(face, (128, 128))  # Resize the face image to 128x128 pixels
            face = np.reshape(face, (1, 128, 128, 1)) / 255.0  # Reshape and normalize the face image for the CNN input
            prediction = model.predict(face)  # Make a prediction using the CNN model
            person_index = np.argmax(prediction)  # Get the index of the predicted class
            confidence = np.max(prediction)  # Get the confidence score of the prediction
            if confidence > 0.7:  # Check if the confidence score is above a threshold
                name = label_names[person_index]  # Get the name of the predicted person
                attendance_data = attendance_data.append({'Name': name, 'Time': datetime.now()}, ignore_index=True)  # Add the attendance record to the DataFrame
                print("Attendance marked for:", name)  # Print the name for whom attendance was marked
            else:
                print("Unknown face detected.")  # Print a message for an unknown face
    elif option == '3':
        quit_program = True  # Set the flag to quit the program
    else:
        print("Invalid option. Please choose again.")  # Print an error message for an invalid option

# Save attendance data to CSV
attendance_data.to_csv('attendance.csv', index=False)  # Save the attendance data to a CSV file

cap.release()  # Release the webcam
cv2.destroyAllWindows()  # Close any open windows

Epoch 1/10
8/8 [==============================] - 5s 565ms/step - loss: 2.5669 - accuracy: 0.3080 - val_loss: 1.7831 - val_accuracy: 0.6333
Epoch 2/10
8/8 [==============================] - 4s 458ms/step - loss: 1.3655 - accuracy: 0.7806 - val_loss: 0.7984 - val_accuracy: 0.9333
Epoch 3/10
8/8 [==============================] - 4s 467ms/step - loss: 0.7387 - accuracy: 0.8776 - val_loss: 0.5319 - val_accuracy: 0.9500
Epoch 4/10
8/8 [==============================] - 4s 465ms/step - loss: 0.4732 - accuracy: 0.8819 - val_loss: 0.5132 - val_accuracy: 0.9333
Epoch 5/10
8/8 [==============================] - 4s 480ms/step - loss: 0.3338 - accuracy: 0.9325 - val_loss: 0.4660 - val_accuracy: 0.9333
Epoch 6/10
8/8 [==============================] - 4s 485ms/step - loss: 0.2416 - accuracy: 0.9367 - val_loss: 0.5655 - val_accuracy: 0.9333
Epoch 7/10
8/8 [==============================] - 4s 448ms/step - loss: 0.1558 - accuracy: 0.9620 - val_loss: 0.5862 - val_accuracy: 0.9333
Epoch 8/10
8/8 [====

C:\Users\Narthana\AppData\Local\Temp\ipykernel_6428\4102362560.py:113: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance_data = attendance_data.append({'Name': name, 'Time': datetime.now()}, ignore_index=True)  # Add the attendance record to the DataFrame


Attendance marked for: narthana
Choose an option: 1 - Register a new face, 2 - Mark attendance, 3 - Quit: 3


In [3]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))  # Train the model on the training data and validate on the testing data

# Get the final accuracy from the history
final_accuracy = history.history['val_accuracy'][-1]
print("Final Validation Accuracy:", final_accuracy)


Epoch 1/10
8/8 [==============================] - 4s 511ms/step - loss: 0.0658 - accuracy: 0.9747 - val_loss: 0.8965 - val_accuracy: 0.9333
Epoch 2/10
8/8 [==============================] - 4s 475ms/step - loss: 0.0343 - accuracy: 0.9958 - val_loss: 0.8444 - val_accuracy: 0.9333
Epoch 3/10
8/8 [==============================] - 4s 451ms/step - loss: 0.0206 - accuracy: 0.9958 - val_loss: 0.9630 - val_accuracy: 0.9333
Epoch 4/10
8/8 [==============================] - 4s 455ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 1.0031 - val_accuracy: 0.9333
Epoch 5/10
8/8 [==============================] - 4s 451ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 1.0336 - val_accuracy: 0.9333
Epoch 6/10
8/8 [==============================] - 4s 516ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 1.0913 - val_accuracy: 0.9333
Epoch 7/10
8/8 [==============================] - 4s 537ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 1.1572 - val_accuracy: 0.9333
Epoch 8/10
8/8 [====